In [25]:
import os
import locale
import math

import pandas as pd
import numpy as np

from datetime import datetime
from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc

import pypdf
from pathlib import Path

from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

from docx2pdf import convert

In [3]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

In [4]:
def get_file_path(row):
    return Path('/media/informacoes_pdf/') / row['setor'].strip() / row['arquivo']


In [5]:
def get_pdf_text(row):
    arquivo = get_file_path(row)
    print(f'File {arquivo} to pypdf text')
    try:
        pdf = pypdf.PdfReader(arquivo)
        text = []
        for page in pdf.pages:
            text.append(page.extract_text())
        return ' '.join(text)
    except FileNotFoundError:
        print(f'File not found: {arquivo}')
        return ''

In [6]:
antecedentes_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

/tmp/ipykernel_160702/146702693.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_antecedentes = pd.read_sql(antecedentes_sql, conn)


In [7]:
processos_antecedentes

,numero_processo,ano_processo,assunto,interessado,marcador
0,000761,2022,APURAÇÃO DE RESPONSABILIDADE EM ATENDIMENTO AO...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,Antecedentes
1,000913,2023,COMUNICAÇÃO DE IRREGULARIDADE ...,SIGILOSO ...,Antecedentes
2,001749,2019,"CONTAS DO CHEFE DO PODER EXECUTIVO MUNICIPAL, ...",PREFEITURA MUNICIPAL DE VIÇOSA ...,Antecedentes
3,002923,2024,CONTAS DO CHEFE DO PODER EXECUTIVO DA PREFEITU...,PREFEITURA MUNICIPAL DE PILÕES ...,Antecedentes
4,003277,2020,AUMENTO REMUNERATÓRIO DURANTE A PANDEMIA - REM...,CÂMARA MUNICIPAL DE ANTÔNIO MARTINS ...,Antecedentes
5,003980,2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,Antecedentes
6,004072,2021,MONITORAMENTO ...,AGÊNCIA DE FOMENTO DO RN S/A ...,Antecedentes
7,004728,2022,DENÚNCIA ...,FRANCINALDO MOREIRA DA SILVA ...,Antecedentes
8,005119,2021,APURAÇÃO DE RESPONSABILIDADE REFERENTE A ATRAS...,SECRETARIA MUNICIPAL DE MEIO AMBIENTE E URBANI...,Antecedentes
9,005181,2020,AUDITORIA DE CONFORMIDADE-EXERCÍCIO 2018 ...,PREFEITURA MUNICIPAL DE MOSSORÓ ...,Antecedentes


In [8]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.vw_Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;


'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

/tmp/ipykernel_160702/3946558724.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)


5    
Name: texto_pdf, dtype: object

In [73]:
def get_transito(nome):
    with open("consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    with get_connection() as conn:
        q = query.format(nome=nome)
        #print(q)
        df = pd.read_sql(q, conn)
        df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df.fillna('', inplace=True)
        return df.to_dict(orient='records')
    
def get_processos_transito_by_cpf(pessoas_cpfs):
    transitos = []
    for pessoa in pessoas_cpfs:
        if pessoa['nome']:
            transitos.append({'nome': pessoa['nome'], 'cpf': pessoa['cpf'], 'transitos': get_transito(pessoa['nome'])})
    return transitos   

def get_genpessoa(pessoa_strc):
    cpfs = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        sql = f'''
        SELECT gp.Documento
            FROM Processo.dbo.GenPessoa gp 
            WHERE lower(gp.nome) = '{nome_pessoa.lower()}'
        '''
        ret = pd.read_sql(sql, get_connection())
        if ret.shape[0] == 0 or ret.shape[0] > 1:
            cpfs.append({'nome': nome_pessoa, 'cpf': None})
        else:
            cpfs.append({'nome': nome_pessoa, 'cpf': ret.iloc[0]['Documento']})
    return cpfs

def is_vazio(valores):
    return all([not bool(v['transitos']) for v in valores])
    

In [74]:
df_info_gaana['texto_pdf'] = df_info_gaana.apply(get_pdf_text, axis=1)

File /media/informacoes_pdf/GAANA/GAANA_000761_2022_0028.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_000913_2023_0027.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_001749_2019_0042.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_002923_2024_0025.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003277_2020_0124.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_003980_2022_0029.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004072_2021_0039.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004728_2022_0049.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005119_2021_0016.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_005181_2020_0084.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_006621_2015_0037.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_200086_2023_0045.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_200123_2023_0038.pdf to pypdf text


In [75]:
llm = AzureChatOpenAI(model_name="gpt-4o")

In [76]:
from typing import List

from pydantic import BaseModel, Field

class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")


In [77]:
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)

In [78]:
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [79]:
df_info_gaana['cpf'] = df_info_gaana['pessoas'].apply(get_genpessoa)

/tmp/ipykernel_160702/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_160702/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_160702/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ret = pd.read_sql(sql, get_connection())
/tmp/ipykernel_160702/2334643007.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlit

In [80]:
df_info_gaana['valores'] = df_info_gaana['cpf'].apply(get_processos_transito_by_cpf)

/tmp/ipykernel_160702/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_160702/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_160702/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_160702/2334643007.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are no

In [81]:
df_info_gaana[df_info_gaana['processo'].str.contains('3980')]['texto_pdf']

5                 \n \n1 \n \n \nProcesso nº: 3980/...
Name: texto_pdf, dtype: object

In [82]:
df_info_gaana[df_info_gaana['processo'].str.contains('3980')]

,setor,processo,assunto,interessado,arquivo,datapublicacao,SequencialProcessoEvento,numero_processo,ano_processo,rn,texto_pdf,pessoas,cpf,valores,len_valores
5,GAANA,003980/2022,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE CEARÁ-MIRIM ...,GAANA_003980_2022_0029.pdf,2025-01-28 12:49:37.590,42,003980,2022,1,\n \n1 \n \n \nProcesso nº: 3980/...,pessoas=[PessoaAntecedentes(nome_pessoa='José ...,"[{'nome': 'José Walter Fernandes', 'cpf': '413...","[{'nome': 'José Walter Fernandes', 'cpf': '413...",3


In [84]:
df_info_gaana['len_valores'] = df_info_gaana['valores'].apply(lambda x: len(x))
antecedentes = df_info_gaana[df_info_gaana['len_valores'] > 0]

In [88]:
for row in antecedentes.itertuples():
    doc = DocxTemplate("templates/antecedentes.docx")
    #valores = [v for v in row.valores if len(v['transitos']) > 0]
    nomes = [v['nome'] for v in row.valores]
    if len(nomes) > 1:
        responsaveis = ', '.join(nomes[:-1]) + ' e ' + nomes[-1]
    else:
        responsaveis = nomes[0] if nomes else ''
    for v in row.valores:
        if v['cpf'] is None:
            v['cpf'] = ''
    context = {
        'processo': row.processo,
        'assunto': row.assunto,
        'interessado': row.interessado,
        'valores': row.valores,
        'responsaveis': responsaveis,
        'data': datetime.now().strftime("%d/%m/%Y"),
    }
    processo_underline = row.processo.replace('/', '_')

    doc.render(context)
    doc.save(f"saidas/antecedentes/{processo_underline}.docx")
    generate_pdf(f"saidas/antecedentes/{processo_underline}.docx", f"saidas/antecedentes/")
    print(f"Gerado arquivo para o processo {row.processo}")

Gerado arquivo para o processo 000761/2022
Gerado arquivo para o processo 000913/2023
Gerado arquivo para o processo 001749/2019
Gerado arquivo para o processo 002923/2024
Gerado arquivo para o processo 003277/2020
Gerado arquivo para o processo 003980/2022
Gerado arquivo para o processo 004072/2021
Gerado arquivo para o processo 004728/2022
Gerado arquivo para o processo 005119/2021
Gerado arquivo para o processo 005181/2020
Gerado arquivo para o processo 006621/2015
Gerado arquivo para o processo 200086/2023
Gerado arquivo para o processo 200123/2023


In [37]:
antecedentes['valores']

1     [{'nome': 'João Silva', 'cpf': None, 'transito...
4     [{'nome': 'Carlos Silva', 'cpf': None, 'transi...
5     [{'nome': 'João Santos', 'cpf': None, 'transit...
6     [{'nome': 'João Silva', 'cpf': None, 'transito...
7     [{'nome': 'José Silva', 'cpf': None, 'transito...
10    [{'nome': 'José da Silva', 'cpf': None, 'trans...
11    [{'nome': 'Carlos Eduardo Santos', 'cpf': None...
Name: valores, dtype: object

In [39]:
valores = antecedentes['valores'].values.tolist()

In [40]:
valores[0]

[{'nome': 'João Silva', 'cpf': None, 'transitos': []},
 {'nome': 'Maria Oliveira', 'cpf': '09831385403', 'transitos': []},
 {'nome': 'Carlos Eduardo',
  'cpf': None,
  'transitos': [{'processo_origem': '014466/2003',
    'processo_execucao': '001370/2014',
    'tipo_debito': 'Multa',
    'valor_original': '1.200,00',
    'valor_atualizado': '1.821,93',
    'transito_julgado': '13/05/2013',
    'situacao_divida': 'Pago Integralmente',
    'status_protesto': '',
    'status_pge': 'Inscrito em Dívida Ativa'},
   {'processo_origem': '017587/2009',
    'processo_execucao': '',
    'tipo_debito': 'Ressarcimento ',
    'valor_original': '616,19',
    'valor_atualizado': '1.113,95',
    'transito_julgado': '11/09/2017',
    'situacao_divida': 'Pago parcialmente',
    'status_protesto': '',
    'status_pge': ''},
   {'processo_origem': '017587/2009',
    'processo_execucao': '',
    'tipo_debito': 'Multa Percentual',
    'valor_original': '308,10',
    'valor_atualizado': '556,98',
    'transit

In [42]:
row.valores

[{'nome': 'Carlos Eduardo Santos', 'cpf': None, 'transitos': []},
 {'nome': 'Ana Beatriz Silva', 'cpf': None, 'transitos': []},
 {'nome': 'Mariana Lima Costa', 'cpf': None, 'transitos': []}]

In [45]:
row

Pandas(Index=11, setor='GAANA', processo='200086/2023', assunto='INADIMPLÊNCIA NA REMESSA DE SIAI DP NOVO                                                                                                                                                                                                                       ', interessado='CÂMARA MUNICIPAL DE AFONSO BEZERRA                                                                                                                                                                                                                             ', arquivo='GAANA_200086_2023_0045.pdf', datapublicacao=Timestamp('2025-04-14 08:47:04.073000'), SequencialProcessoEvento=74, numero_processo='200086', ano_processo='2023', rn=1, texto_pdf='', pessoas=PessoasAntecedentes(pessoas=[PessoaAntecedentes(nome_pessoa='Carlos Eduardo Santos'), PessoaAntecedentes(nome_pessoa='Ana Beatriz Silva'), PessoaAntecedentes(nome_pessoa='Mariana Lima Costa')]), cpf=[{'nome':